In [8]:
import urllib2
response = urllib2.urlopen("http://www.pythonscraping.com/pages/page3.html")
html = response.read()
#print(html)

In [10]:
from bs4 import BeautifulSoup

bsObj = BeautifulSoup(html)

for sibling in bsObj.find("table",{"id":"giftList"}).tr.next_siblings:
    print(sibling) 



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

In [13]:
import re

images = bsObj.findAll("img", {"src":re.compile("\.\.\/img\/gifts/img.*\.jpg")})
for image in images: 
    print(image["src"])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


### WikiPedia by Random

In [4]:
import urllib2
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    res = urllib2.urlopen("http://de.wikipedia.org"+articleUrl)
    html = res.read()
    bsObj = BeautifulSoup(html)
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$"))

links = getLinks("/wiki/Basel")
while len(links) > 15:
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)


/wiki/Basler_Kantonalbank
/wiki/Kantonalbank
/wiki/%C3%96ffentlich-rechtliches_Kreditinstitut
/wiki/Europ%C3%A4ische_Kommission
/wiki/Simone_Veil
/wiki/Jurist
/wiki/Juristenausbildung_in_der_Schweiz


### get Country

In [6]:
import json
import urllib2


def getCountry(ipAddress):
    response = urllib2.urlopen("http://freegeoip.net/json/"+ipAddress).read().decode('utf-8')
    responseJson = json.loads(response)
    return responseJson.get("country_code")

print(getCountry("50.78.253.58"))
print(getCountry("131.152.1.1"))

US
CH


### Wiki History

In [11]:
import urllib2
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    res = urllib2.urlopen("http://de.wikipedia.org"+articleUrl)
    html = res.read()
    bsObj = BeautifulSoup(html)
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a", 
                     href=re.compile("^(/wiki/)((?!:).)*$"))

def getHistoryIPs(pageUrl):
    #Format of revision history pages is: 
    #http://en.wikipedia.org/w/index.php?title=Title_in_URL&action=history
    pageUrl = pageUrl.replace("/wiki/", "")
    historyUrl = "http://de.wikipedia.org/w/index.php?title="+pageUrl+"&action=history"
    print("history url is: "+historyUrl)
    res = urllib2.urlopen(historyUrl)
    html = res.read()
    bsObj = BeautifulSoup(html)
    #finds only the links with class "mw-anonuserlink" which has IP addresses 
    #instead of usernames
    ipAddresses = bsObj.findAll("a", {"class":"mw-anonuserlink"})
    addressList = set()
    for ipAddress in ipAddresses:
        addressList.add(ipAddress.get_text())
    return addressList


def getCountry(ipAddress):
    try:
        response = urllib2.urlopen("http://freegeoip.net/json/"
                           +ipAddress).read().decode('utf-8')
    except HTTPError:
        return None
    responseJson = json.loads(response)
    return responseJson.get("country_code")
    
links = getLinks("/wiki/Basel")


while(len(links) > 7):
    for link in links:
        print("-------------------") 
        historyIPs = getHistoryIPs(link.attrs["href"])
        for historyIP in historyIPs:
            country = getCountry(historyIP)
            if country is not None:
                print(historyIP+" is from "+country)

    newLink = links[random.randint(0, len(links)-1)].attrs["href"]
    links = getLinks(newLink)

-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Basel_(Begriffskl%C3%A4rung)&action=history
79.246.164.59 is from DE
129.132.211.197 is from CH
91.6.246.237 is from DE
88.72.49.214 is from DE
88.72.33.85 is from DE
217.224.254.59 is from DE
85.1.163.19 is from CH
217.230.59.78 is from DE
-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Schweiz&action=history
-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Kanton_(Schweiz)&action=history
84.74.177.35 is from CH
2a02:908:fd32:9900:651f:1bbd:8e48:668 is from DE
92.106.225.204 is from CH
5.149.18.72 is from CH
149.172.245.126 is from DE
-------------------
history url is: http://de.wikipedia.org/w/index.php?title=Kanton_Basel-Stadt&action=history
178.196.243.56 is from CH
194.153.189.154 is from CH
94.198.248.247 is from CH
83.76.228.210 is from CH
92.76.199.49 is from DE
2002:b253:b780:0:3040:7c5c:a696:9e6e is from CH
109.90.123.147 is from DE
-------

NameError: global name 'HTTPError' is not defined

## HTML Parser

In [1]:
from HTMLParser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)
    def handle_endtag(self, tag):
        print("Encountered an end tag :", tag)
    def handle_data(self, data):
        print("Encountered some data  :", data)

parser = MyHTMLParser()
parser.feed('<html><head><title>Test</title></head>'
            '<body><h1>Parse me!</h1></body></html>')

('Encountered a start tag:', 'html')
('Encountered a start tag:', 'head')
('Encountered a start tag:', 'title')
('Encountered some data  :', 'Test')
('Encountered an end tag :', 'title')
('Encountered an end tag :', 'head')
('Encountered a start tag:', 'body')
('Encountered a start tag:', 'h1')
('Encountered some data  :', 'Parse me!')
('Encountered an end tag :', 'h1')
('Encountered an end tag :', 'body')
('Encountered an end tag :', 'html')


In [7]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)
    def handle_endtag(self, tag):
        print("Encountered an end tag :", tag)
    def handle_data(self, data):
        print("Encountered some data  :", data)

res = urllib2.urlopen("http://www.nzz.ch")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

('Encountered some data  :', u' ')
('Encountered a start tag:', u'html')
('Encountered some data  :', u' ')
('Encountered a start tag:', u'head')
('Encountered a start tag:', u'meta')
('Encountered an end tag :', u'meta')
('Encountered some data  :', u' ')
('Encountered a start tag:', u'title')
('Encountered some data  :', u'NZZ News \u2013 aktuelle Nachrichten, Hintergr\xfcnde, Kommentare')
('Encountered an end tag :', u'title')
('Encountered some data  :', u' ')
('Encountered a start tag:', u'script')
('Encountered some data  :', u"(function(H){H.className=H.className.replace(/\\bno-js\\b/,'js ')})(document.documentElement)")
('Encountered an end tag :', u'script')
('Encountered some data  :', u' ')
('Encountered a start tag:', u'script')
('Encountered some data  :', u'\n {\n "@context" : "http://schema.org",\n "@type" : "Organization",\n "name" : "NZZ Neue Z\xfcrcher Zeitung",\n "url" : "http://www.nzz.ch",\n "logo" : "http://www.nzz.ch/nzz.png",\n "sameAs" : [ "https://www.facebook

In [11]:

from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Encountered a start tag:", tag)
        for attr in attrs:
            print("Attr", attr)
            
#    def handle_endtag(self, tag):
        #print("Encountered an end tag :", tag)
#    def handle_data(self, data):
        #print("Encountered some data  :", data)

res = urllib2.urlopen("http://www.nzz.ch")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)



('Encountered a start tag:', u'html')
('Attr', (u'lang', u'de'))
('Attr', (u'class', u'no-js '))
('Encountered a start tag:', u'head')
('Encountered a start tag:', u'meta')
('Attr', (u'http-equiv', u'X-UA-Compatible'))
('Attr', (u'content', u'IE=edge'))
('Encountered a start tag:', u'title')
('Encountered a start tag:', u'script')
('Encountered a start tag:', u'script')
('Attr', (u'type', u'application/ld+json'))
('Encountered a start tag:', u'meta')
('Attr', (u'http-equiv', u'content-type'))
('Attr', (u'content', u'text/html; charset=utf-8'))
('Encountered a start tag:', u'link')
('Attr', (u'rel', u'dns-prefetch'))
('Attr', (u'href', u'//assets.static-nzz.ch'))
('Encountered a start tag:', u'link')
('Attr', (u'rel', u'dns-prefetch'))
('Attr', (u'href', u'//api.adquality.ch'))
('Encountered a start tag:', u'link')
('Attr', (u'rel', u'dns-prefetch'))
('Attr', (u'href', u'//nexus.ensighten.com'))
('Encountered a start tag:', u'link')
('Attr', (u'rel', u'dns-prefetch'))
('Attr', (u'href',

In [10]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    
    catch = 0
    catchline = ""
    
    def handle_starttag(self, tag, attrs):
        #print("Encountered a start tag:", tag)
        for attr in attrs:
            #print("Attr", attr)
            #print(attr[1])
            if attr[1] == "title__catchline":
                self.catch = 2
            elif attr[1] == "title__name":
                self.catch = 1
            else:
                self.catch = 0
            
#    def handle_endtag(self, tag):
        #print("Encountered an end tag :", tag)
    
    def handle_data(self, data):
        if self.catch==1:
            print self.catchline+": "+data
            self.catchline = ""
            
        if self.catch==2:
            self.catchline = data

res = urllib2.urlopen("http://www.nzz.ch")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

Zwischen Goldstandard und EZB: Wem gehört der Euro?
Sparprogramm im Parlament: Streiks und Proteste in Griechenland
: Was heute wichtig ist
68. Filmfestival Locarno: Ein Programm rund um Häuser
Kampf gegen Boko Haram: Der zögerliche Präsident
Korruptionsverdacht: «Boxenstopp» für den Chef der Tessiner Steuerkontrolle
Auschwitz-Prozess in Lüneburg: Vier Jahre Haft für früheren SS-Mann Gröning
«Wirtschaft im Fokus» mit Marc Parmentier: «Wer gut schlafen will, sollte Festhypotheken abschliessen»
Philosophie des Anti-Aging: Was, wenn wir vor lauter Langeweile sterben möchten?
Amerikanische Esskultur in der Schweiz: Schöne neue Konsumwelt
Vor der Abstimmung im griechischen Parlament: Tsipras gibt sich kämpferisch
Griechische Schuldenlast nicht tragbar: Tiefer Graben zwischen dem IMF und Europa
Griechenland-Deal: Das hätte man billiger haben können
Seitenblick: Im Umgang mit Rettungsschwimmern
Parteipolitische Kakofonie: Adieu, Konkordanz
Vereinbarung mit Iran: Eine Atempause, keine Atompaus

## Rubriken NZZ

In [51]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    
    rubrik = 0
    catch = 0
    tmp = 0
    
    
    def handle_starttag(self, tag, attrs):
        #print("Encountered a start tag:", tag)
        for attr in attrs:
            #print("Attr", attr)
            #print(attr[1])
            if attr[1]!=None and attr[1].find("container__title ")!=-1 :
                #print("Rubrik")
                self.rubrik += 1
                self.catch = 1
            else:
                #self.catch = 0
                self.tmp=0
    
    def handle_endtag(self, tag):
        if tag == "a" or tag == "div":
            self.catch=0
    
    def handle_data(self, data):
        if self.catch==1:
            if len(data) > 2:
                print(str(self.rubrik)+". "+data)
            

res = urllib2.urlopen("http://www.nzz.ch")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

1. Entscheidungstage für Griechenland
2. Meinung und Debatte
3. International
4. Wirtschaft
5. Finanzen
6. Schweiz
7. NZZ-Spezial zu den Wahlen 2015
8. Feuilleton
9. Zürich
10. Sport
11. Digital
12. Video
13. Panorama
14. Lebensart
15. Medien
16. Wissenschaft
17. Aktuelle Bildstrecken
18. Bild des Tages


###Find Last Article

In [52]:
from HTMLParser import HTMLParser
import urllib2

class MyHTMLParser(HTMLParser):
    
    newesttime = 0
    title = ""
    tmp = 0
    catch=0
    
    
    def handle_starttag(self, tag, attrs):
        #print("Encountered a start tag:", tag)
        if tag == "div":
            for attr in attrs:
                if attr[1] == "title__name":
                    self.catch = 1
            
        
        if tag == "time":
            for attr in attrs:
            #print("Attr", attr)
            #print(attr[1])
                if attr[1]!=None and attr[0].find("datetime")!=-1 :
                    print attr[1]
                

    
    def handle_endtag(self, tag):
        if tag == "a" or tag == "div":
            self.catch=0
    
    def handle_data(self, data):
        if self.catch==1:
            if len(data) > 2:
                print(data)
            

res = urllib2.urlopen("http://www.nzz.ch")        
encoding = res.headers.getparam('charset')
html = res.read().decode(encoding)             
parser = MyHTMLParser()
parser.feed(html)

Wem gehört der Euro?
2015-07-15 05:30:00
Garstiges Umfeld für Sparer und Anleger
2015-07-15 05:30:00
Was heute wichtig ist
2015-07-15 09:40:00
Piccards Solar Impulse fliegt erst 2016 wieder
2015-07-15 14:18:00
Fehdehandschuh der schottischen Nationalisten
2015-07-15 14:39:00
Vier Jahre Haft für früheren SS-Mann Gröning
2015-07-15 10:58:00
Streiks und Proteste in Griechenland
2015-07-15 12:33:00
«Boxenstopp» für den Chef der Tessiner Steuerkontrolle
2015-07-15 13:24:00
2015-07-15 14:43:41
2015-07-15 14:31:11
2015-07-15 14:28:46
2015-07-15 14:21:17
2015-07-15 14:19:59
Neue Hitzewelle gefährdet Feuerwerke am 1. August
2015-07-15 04:00:00
«Wer gut schlafen will, sollte Festhypotheken abschliessen»
2015-07-15 06:00:00
Was, wenn wir vor lauter Langeweile sterben möchten?
2015-07-15 05:30:00
Tsipras gibt sich kämpferisch
2015-07-15 08:10:00
Tiefer Graben zwischen dem IMF und Europa
2015-07-15 05:17:00
Das hätte man billiger haben können
2015-07-14 21:32:00
Im Umgang mit Rettungsschwimmern
201